Mount Google Drive



In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft "trl<0.15.0" triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.2.7 requires tyro, which is not installed.
unsloth-zoo 2025.2.7 requires protobuf<4.0.0, 

In [ ]:
import os
import wandb

# Yêu cầu người dùng nhập Wandb API key
wandb_api_key = input("Nhập Wandb API key của bạn: ")

# Thiết lập biến môi trường WANDB_API_KEY
os.environ["WANDB_API_KEY"] = wandb_api_key

# Đăng nhập vào wandb
wandb.login()

wandb: Currently logged in as: tuanhoangtc (tuanhoangtc-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(
    project="IELTS_Feedback",  # Tên dự án trên WandB
    name="Run_4_test"  # Tên cho lần chạy này
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Meta-Llama-3.1-8B",
    model_name = "unsloth/Llama-3.2-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Loading libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
from datasets import Dataset

In [ ]:
from huggingface_hub import login

# Yêu cầu nhập token từ người dùng
token = input("Nhập token Hugging Face của bạn: ")

# Thực hiện đăng nhập
login(token)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ielts dataset/slow_module_dataset_5.csv')
print(len(data))

8361


In [ ]:
PROMPT = """
In this task, you are required to evaluate an IELTS Writing Task 2 essay. Consider the following criterias and provide a detailed assessment for each:

- Task Achievement:
+ Evaluate if the candidate fully addresses all parts of the task.
+ Assess the clarity of the thesis statement and position.
+ Check if arguments are developed with specific examples/data (e.g., studies, real-world cases).
+ Identify generalizations (e.g., "some people believe") and suggest replacements with evidence.

- Coherence and Cohesion:
+ Assess paragraph structure: Does each paragraph focus on one central idea?
+ Evaluate the variety and accuracy of linking devices (e.g., "however" for contrast, "for instance" for examples).
+ Check if the conclusion logically summarizes key points without introducing new ideas.

- Coherence and Cohesion:
+ Identify repetitive words/phrases and suggest synonyms (e.g., "important" → "crucial," "significant").
+ Highlight unnatural collocations (e.g., "make an interview" → "conduct an interview").
+ Assess the use of topic-specific vocabulary (e.g., "cognitive bias," "behavioral assessment").

- Grammatical Range and Accuracy:
+ List grammatical errors (e.g., subject-verb agreement, tense misuse) and provide corrected versions.
+ Evaluate the use of complex structures (e.g., relative clauses, passive voice).
+ Check punctuation (e.g., comma splices, missing apostrophes).

- Feedback and Additional Comments:
+ Highlight 2–3 strengths (e.g., "effective use of examples") and 3–4 actionable improvements (e.g., "practice using relative clauses").
+ Provide practice suggestions (e.g., "Write 5 sentences using collocations related to employment").

- Prompt:
{}

- Essay:
{}

- Evaluation:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    prompt = examples["prompt"]
    essay  = examples["essay"]
    evaluation = examples["evaluation"]
    texts = []
    for prompt, essay, evaluation in zip(prompt, essay, evaluation):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = PROMPT.format(prompt, essay, evaluation) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_train = Dataset.from_pandas(data)
dataset_train = dataset_train.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/8361 [00:00<?, ? examples/s]

In [ ]:
data2 = pd.read_csv('/content/drive/MyDrive/ielts dataset/test_slow_module_dataset_4.csv')
data2

,prompt,essay,evaluation
0,Interview form the basic selection criteria fo...,To agree or disagree with the statement that i...,- Task Achievement :\n+ The essay effectively ...
1,The increase in the production of consumer con...,It is true that the rise in consumer productio...,- Task Achievement:\n+ The essay adequately ad...
2,The increase in the production of consumer goo...,It is widely argued that the growth of the num...,- Task Achievement:\n+ The essay effectively a...
3,The increase in the production of consumer goo...,The production of new items and objects have b...,- Task Achievement:\n+ The candidate has parti...
4,The increase in the production of consumer goo...,"In recent years, an increase in the manufactur...",- Task Achievement :\n+ The essay effectively ...
...,...,...,...
486,"As well as making money, businesses also have ...",The role of companies is to produce all the go...,- Task Achievement :\n+ The essay effectively ...
487,"As well as making money, businesses also have ...",Although earning money is one of the most impo...,- Task Achievement:\n+ The essay adequately ad...
488,"As well as making money, businesses also have ...",IIt is argued by many people that companies ha...,- Task Achievement :\n+ The essay addresses th...
489,"As well as making money, businesses also have ...","In this modern era , everyone needs money for ...",- Task Achievement:\n+ The candidate has adequ...


In [ ]:
dataset_test = Dataset.from_pandas(data2)
dataset_test = dataset_test.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/491 [00:00<?, ? examples/s]

In [ ]:
dataset_train

Dataset({
    features: ['prompt', 'essay', 'evaluation', 'text'],
    num_rows: 8361
})

In [ ]:
dataset_test

Dataset({
    features: ['prompt', 'essay', 'evaluation', 'text'],
    num_rows: 491
})

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_test,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 5,
        max_steps = 40,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.05,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/MyDrive/ielts_feedback_outputs",
        report_to = "wandb", # Use this for WandB etc
        # evaluation_strategy="epoch",
    ),
)

Map (num_proc=2):   0%|          | 0/8361 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/491 [00:00<?, ? examples/s]

In [ ]:
!pip install timeout-decorator

  Preparing metadata (setup.py) ... done
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5007 sha256=533d5507775ffca4b6994f885551d656444f14e08ff500357a27a9ff4ba249c2
  Stored in directory: /root/.cache/pip/wheels/aa/cd/d1/51736c6b95846b2613a520ce146a8f305c4016a987bc9faec7
Successfully built timeout-decorator


In [ ]:
import timeout_decorator

@timeout_decorator.timeout(600)  # 120 giây = 2 phút
def run_training():
    return trainer.train()

try:
    trainer_stats = run_training()
except timeout_decorator.TimeoutError:
    print("Training mất quá 5 phút, chuyển sang cell tiếp theo.")
    trainer.save_model(trainer.args.output_dir)

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e6d08ff841f2437f253277b0280545782ad169da0b5669779a9f87497a9bb2f7
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-z4crlieq
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-z4crlieq
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=f89dd0e71e4ef4316a36b54c7a64880caf29155996c20801ae44867cffd46f5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-11blp8pe/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
test_predictions = []

for i in range(0,10):
  print(i)
  prompt = dataset['test'][i]['prompt']
  essay = dataset['test'][i]['essay']
  inputs = tokenizer(
  [
    PROMPT.format(
        prompt,
        essay,
        "",
    )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, temperature=0.8, do_sample=True)
  answer = tokenizer.batch_decode(outputs)
  evaluation = answer[0].split("Evaluation:")[1]

  test_predictions.append(evaluation)

0
1
2
3
4
5
6
7
8
9


In [ ]:
# prompt: load file json from google drive

import json

def load_json_from_google_drive(file_path):
  with open(file_path, 'r') as f:
    data = json.load(f)
  return data

# Example usage
file_path = '/content/drive/MyDrive/test_predictions.json' # Replace with your file path
test_predictions = load_json_from_google_drive(file_path)

test_references = load_json_from_google_drive('/content/drive/MyDrive/test_references.json')

In [ ]:
from evaluate import load

# Load các metric
bleu = load("bleu")
rouge = load("rouge")
meteor = load("meteor")
bleurt = load("bleurt", module_type="metric", checkpoint="BLEURT-20")  # BLEUScore
bertscore = load("bertscore")

# Chuyển đổi định dạng của dữ liệu
# references = [[ref] for ref in test_references[:10]]  # Mỗi reference phải nằm trong một list

# Tính BLEU
bleu_result = bleu.compute(predictions=test_predictions, references=test_references)
print("BLEU:", bleu_result["bleu"])

# Tính ROUGE
rouge_result = rouge.compute(predictions=test_predictions, references=test_references)
print("ROUGE-1:", rouge_result["rouge1"])
print("ROUGE-2:", rouge_result["rouge2"])
print("ROUGE-L:", rouge_result["rougeL"])

# Tính METEOR
meteor_result = meteor.compute(predictions=test_predictions, references=test_references)
print("METEOR:", meteor_result["meteor"])

# Tính BLEUScore
bleurt_result = bleurt.compute(predictions=test_predictions, references=test_references)
bertScore = 0
for i in bleurt_result["scores"]:
  bertScore += i
bertScore = bertScore/len(bleurt_result["scores"])
print("BLEURTScore:", bertScore)

bertscore_result = bertscore.compute(predictions=test_predictions, references=test_references, model_type="microsoft/deberta-large-mnli")
bert_score = 0
for i in bertscore_result["precision"]:
  bert_score += i
bert_score = bert_score/len(bertscore_result["precision"])
print("BERTScore:", bert_score)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU: 0.15522185697869428
ROUGE-1: 0.450232814796871
ROUGE-2: 0.20781333688337
ROUGE-L: 0.27881554388980834
METEOR: 0.36861452051589416
BLEURTScore: -0.048542015999555585


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

BERTScore: 0.6363051891326904


In [ ]:
prompt = """
In most countries, animal and plant species are declining rapidly. What are the causes of this? What measures could be done to prevent this decline?
"""
essay = """
The species of animals and plants are rapidly decreasing in most countries. In this essay, I will examine the factors that contribute towards declining animal and plant species and propose some solutions to that.

There are a few factors that contribute to the falling species of animals and plants. Firstly, the main cause for this issue probably is human activities. There are some people who like to kill animals and plants for their own purpose such as for collection. Secondly, the other reason why is this happening is because of nature degradation. Nowadays, climate change is getting worse in most countries  leading to natural selection for animals and plants. Therefore, their species witnessed a rapid fallen.

A few ways can be taken to prevent the decline of animal and plant species. Firstly, government plays a crucial role. They have to be aware of this issue and then  formulate some regulations to prevent animal and plant hunting. Although the law is already generated, the government should be more active to make sure the regulation is well-running. Furthermore, as citizens, we can drive a movement to raise awareness about this issue. Maybe, some people will underestimate what can citizens do with this little, but I believe if we hold hands together, we can make a change.

In conclusion, the species of animals and plants are declining rapidly  mainly caused by human activities that do illegal hunting. This essay suggested that the ways to prevent this problem are twofold: to generate strict regulation and to create a movement in order to raise society's awareness.
"""

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    PROMPT.format(
        prompt,
        essay,
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, temperature=0.8, do_sample=True)
answer = tokenizer.batch_decode(outputs)
evaluation = answer[0].split("Evaluation:")[1]

#print(f"prompt:\n{prompt}")
#print(f"essay: \n{essay}")
print(evaluation)



- Task Achievement:
+ The candidate has adequately addressed the task by discussing the causes and solutions for declining animal and plant species
+ All aspects of the task have been covered with clear and relevant ideas presented
+ The essay effectively meets the requirements of the task and provides a satisfactory response
- Coherence and Cohesion:
+ The essay is generally wellorganized with clear transitions between sentences and paragraphs
+ Connecting words and phrases are used effectively to maintain a smooth progression of ideas
+ The logical flow of information is maintained throughout the essay
- Lexical Resource:
+ The essay demonstrates a reasonable range of vocabulary
+ Some inaccuracies in vocabulary use are evident such as "degrading" for "degrading" and "hunting" for "illegal hunting"
+ More precise and varied vocabulary could enhance the overall quality of the essay
- Grammatical Range and Accuracy:
+ The essay exhibits a variety of sentence structures
+ Punctuation 